In [ ]:
import xspec as xs
import pandas as pd
import os
import numpy as np

##### Column density value don't freeze initially. If the column density estimates are one or more order less than the galactic value then freeze. Also check the significance of the parameters. Save the .xcm file. 

In [ ]:
cwd = os.getcwd()
os.chdir(f'{cwd}')
pd.read_csv('spec_info.csv')

In [ ]:
list_file = pd.read_csv('spec_info.csv')
source_no = list_file['source_no.']
obsids = list_file['obsid']
gal_nh = list_file['avg_galactic_nh'] # Absolute value with unit as 1
grp = list_file['grouping']
z = list_file['redshift']

gal_nh_22 = gal_nh*1e-22 # In units of 10^22


In [ ]:
#Empty array where the result will be stored
nh = []
nh_hi_err = []
nh_lo_err = []

gamma = []
gamma_hi_err = []
gamma_lo_err = []

norm = []
norm_hi_err = []
norm_lo_err = []

flux = []
flux_hi_err = []
flux_lo_err = []

lumin = []
lumin_hi_err = []
lumin_lo_err = []

dof = []
stat = []

n = int(input('How many?'))

for i in range(n):
  
    spec_path = f'{source_no[i]}/{obsids[i]}/repro/spectrum/'
    os.chdir(f'{cwd}/{spec_path}')
#     xs.Xset.openLog('xspec.log')
    if grp[i] != 1 :
        xs.Fit.statMethod = "chi"
    else:
        xs.Fit.statMethod = "cstat"

    #Loading spectra and model
    s = xs.Spectrum('spec_grp.pi')
    m = xs.Model("tbabs*pow", setPars = {1:gal_nh_22[i], 2:2})
    xs.Plot.xAxis = 'keV'
    s.ignore('**-0.5 7.0-**')
    par1 = m.TBabs.nH
    par2 = m.powerlaw.PhoIndex
    par3 = m.powerlaw.norm
   
    
    flag = False
    
    try : 
        xs.Fit.perform()
        par1_value = par1.values[0]
        par2_value = par2.values[0]
        par3_value = par3.values[0]
    except :
#         raise "XSPEC Error:  No variable parameters for fit"
        flag = True
        
  
    #if estimated nh is less than one order of magnitude lower then freeze it
    if par1_value < gal_nh_22[i]/10 or par1_value > 10^2 or flag == True: 

#     if True : # Freezing nH for all  
        m.setPars(f'{gal_nh_22[i]}')
        par1.frozen = True
        xs.Fit.perform()
        par1_value = par1.values[0]
        par2_value = par2.values[0]
        par3_value = par3.values[0]
        
        try :
            xs.Fit.error("2-3")

            #collecting the values and error of the parameters 
            #(Remember these all are not the absolute value rather the values at the given units)
            par1_lo_err = 0
            par1_hi_err = 0

            par2_lo_err = par2_value - par2.error[0]
            par2_hi_err = par2.error[1] - par2_value

            
            par3_lo_err = par3_value - par3.error[0]
            par3_hi_err = par3.error[1] - par3_value
        except :
            par1_lo_err = 0
            par1_hi_err = 0
            par2_lo_err = 'nan'
            par2_hi_err = 'nan'
            par3_lo_err = 'nan'
            par3_hi_err = 'nan'
    else :
        #collecting the values and error of the parameters (Remember these all are not the absolute value rather 
        #the values at the given units)
        try :
            xs.Fit.error("1-3")
            par1_lo_err = par1_value - par1.error[0]
            par1_hi_err = par1.error[1] - par1_value

            par2_lo_err = par2_value - par2.error[0]
            par2_hi_err = par2.error[1] - par2_value

            par3_lo_err = par3_value - par3.error[0]
            par3_hi_err = par3.error[1] - par3_value
        
        except :
            par1_lo_err = 'nan'
            par1_hi_err = 'nan'
            par2_lo_err = 'nan'
            par2_hi_err = 'nan'
            par3_lo_err = 'nan'
            par3_hi_err = 'nan'

    xs.AllModels.calcFlux('0.5 7.0 error')
    flux_i = s.flux

    #store values for the different sources in the array
    nh.append(par1_value*1e+22)
    if np.is_nan()
    nh_hi_err.append(par1_hi_err*1e+22)
    nh_lo_err.append(par1_lo_err*1e+22)

    gamma.append(par2_value)
    gamma_hi_err.append(par2_hi_err)
    gamma_lo_err.append(par2_lo_err)

    norm.append(par3_value)
    norm_hi_err.append(par3_hi_err)
    norm_lo_err.append(par3_hi_err)

    flux.append(flux_i[0])
    flux_lo_err.append(flux_i[0] - flux_i[1])
    flux_hi_err.append(flux_i[2] - flux_i[0])

    stat.append(xs.Fit.statistic)
    dof.append(xs.Fit.dof)
    
    if np.invert(np.isnan(z[i])) :
        xs.AllModels.calcLumin(f'0.5 7.0 {z[i]} error')
        lumin_i = s.lumin
        
        lumin.append(lumin_i[0] * 1e+44)
        lumin_lo_err.append((lumin_i[0] - lumin_i[1])*1e+44)
        lumin_hi_err.append((lumin_i[2] - lumin_i[0])*1e+44)
    
    else :
        lumin.append('nan')
        lumin_lo_err.append('nan')
        lumin_hi_err.append('nan')
        
    
#     xs.Xset.save(fileName = 'tbabs*pow',info ='a')
#     xs.Xset.closeLog()
#     xs.Plot.device = "/xs"
#     xs.Plot.iplot('data','resid')
#     xs.Plot.show()
    xs.AllData.clear()
    os.chdir(f'{cwd}')


In [ ]:
os.chdir(f'{cwd}/')
spec_output = open('spectral_modelling_result_0.5-7kev_test_thaw_nh_for_high_value.csv','a')
#uncomment the following line only if there is no such file and that is for the first time only
spec_output.write('''source_no.,obid,flux,flux_hi_err,flux_lo_err,redshift,lumin,lumin_hi_err,lumin_lo_err,nh,nh_hi_err,nh_lo_err,gamma,gamma_hi_err,gamma_low_err,norm,norm_hi_err,norm_lo_err,stat,dof,red_stat\n''')

#load the output of the modelling to the csv file
for i in range(n):
    print(i)
    spec_output.write(f'{source_no[i]},{obsids[i]},{flux[i]},{flux_hi_err[i]},{flux_lo_err[i]},{z[i]},{lumin[i]},{lumin_hi_err[i]},{lumin_lo_err[i]},{nh[i]},{nh_hi_err[i]},{nh_lo_err[i]},{gamma[i]},{gamma_hi_err[i]},{gamma_lo_err[i]},{norm[i]},{norm_hi_err[i]},{norm_lo_err[i]},{stat[i]},{dof[i]},{stat[i]/dof[i]}\n')

spec_output.close()